In [1]:
# Dependencies to Visualize the model
%matplotlib inline
from IPython.display import Image, SVG
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)
# Filepaths, numpy, and Tensorflow
import os
import numpy as np
import tensorflow as tf
# Sklearn scaling
from sklearn.preprocessing import MinMaxScaler
# Keras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [13]:
from PIL import Image
import glob
def getImages(path):
    image_list = []
    for filename in glob.glob(f"{path}*.jpg"):
        im=Image.open(filename)
        image_list .append(im)
    for filename in glob.glob(f"{path}*.png"):
        im=Image.open(filename)
        image_list.append(im)
    return image_list    

In [21]:
angry = getImages('samples/angry/')
disgust = getImages('samples/disgust/')
fear = getImages('samples/fear/')
happy = getImages('samples/happy/')
neutral = getImages('samples/neutral/')
sad = getImages('samples/sad/')
surprise = getImages('samples/surprise/')